In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from modules.competition_dataset import EEGDataset
import random
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from scipy.linalg import expm, logm
from sklearn.pipeline import make_pipeline
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.mean import mean_riemann
from pyriemann.utils.distance import distance_riemann
from sklearn.base import BaseEstimator, TransformerMixin
from pyriemann.classification import MDM
from sklearn.feature_selection import mutual_info_classif
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from scipy.linalg import logm, inv
from pyriemann.utils.mean import mean_riemann
from scipy.optimize import minimize
from pyriemann.spatialfilters import CSP
from pyriemann.classification import FgMDM
from scipy.signal import sosfiltfilt, butter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


In [ ]:
data_path = './data/mtcaic3'
lda_model_path = './checkpoints/mi/models/lda_mi.pkl'

# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_random_seeds(42)

In [ ]:
window_length = 1000
stride = window_length // 2
batch_size = 64

In [64]:
eeg_channels = [
    "FZ",
    "C3",
    "CZ",
    "C4",
    # "PZ",
    "PO7",
    # "OZ",
    "PO8",
]

dataset_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    task="mi",
    split="train",
    data_fraction=0.4,
    tmin=250,
    eeg_channels=eeg_channels,
)

dataset_val = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    task='mi',
    split='validation',
    read_labels=True,
    tmin=250,
    eeg_channels=eeg_channels,
)

dataset_test = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    task='mi',
    split='test',
    read_labels=False,
    tmin=250,
    eeg_channels=eeg_channels,
)

dataset_train[0][0].shape

task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples
skipped: 16/960
task: mi, split: validation, domain: time, data_fraction: 1.0
skipped: 1/50
task: mi, split: test, domain: time, data_fraction: 1.0
skipped: 0/50


torch.Size([6, 1000])

In [65]:
all_data = torch.cat([torch.stack([x for x,_ in ds]) for ds in (dataset_train, dataset_val, dataset_test)])
X_val_train = torch.cat([torch.stack([x for x,_ in ds]) for ds in (dataset_train, dataset_val)])
y_val_train = torch.cat([torch.stack([y for _,y in ds]) for ds in (dataset_train, dataset_val)])

mean = all_data.mean((0, 2))
std = all_data.std((0, 2))

X_val_train = (X_val_train - mean[None, :, None]) / std[None, :, None]

mean, std

(tensor([0.0235, 0.0523, 0.0713, 0.0441, 0.0258, 0.0490]),
 tensor([0.9722, 0.9950, 1.0636, 1.0019, 0.9947, 0.9900]))

In [66]:
import numpy as np
from sklearn.feature_selection import f_classif

# Concatenate all splits (add dataset_val and dataset_test if needed)
X_all = np.concatenate([
    dataset_train.data.numpy(),
    dataset_val.data.numpy(),
    dataset_test.data.numpy(),
], axis=0)  # shape: [N_total, C, ...]
y_all = np.concatenate([
    dataset_train.labels.numpy(),
    dataset_val.labels.numpy(),
    dataset_test.labels.numpy(),
], axis=0)  # shape: [N_total]

# Detect shape and adapt
if X_all.ndim == 3:
    # [B, C, T]
    num_samples, num_channels, time_points = X_all.shape
    channel_f_scores = []
    for i in range(num_channels):
        channel_data = X_all[:, i, :]  # [N_total, T]
        f_scores_per_timepoint, _ = f_classif(channel_data, y_all)
        aggregated_f_score = np.sum(f_scores_per_timepoint)
        channel_f_scores.append(aggregated_f_score)
elif X_all.ndim == 4:
    # [B, C, F, T]
    num_samples, num_channels, freq_points, time_points = X_all.shape
    channel_f_scores = []
    for i in range(num_channels):
        # Average over freq and time for each channel
        channel_data = X_all[:, i, :, :].mean(axis=(1, 2))  # [N_total]
        f_score, _ = f_classif(channel_data.reshape(-1, 1), y_all)
        channel_f_scores.append(f_score[0])
else:
    raise ValueError(f"Unsupported data shape: {X_all.shape}")

# Optionally, map to channel names
original_channel_names = eeg_channels
channel_scores_dict = {original_channel_names[i]: channel_f_scores[i] for i in range(num_channels)}

print("\n--- F-scores for each channel (higher score indicates more informativeness) ---")
sorted_channels = sorted(channel_scores_dict.items(), key=lambda item: item[1], reverse=True)
for channel, score in sorted_channels:
    print(f"  {channel}: {score:.2f}")

top_3_channels = [channel for channel, score in sorted_channels[:3]]
print(f"\n--- Recommended Top 3 Channels based on F-score: {top_3_channels} ---")


--- F-scores for each channel (higher score indicates more informativeness) ---
  CZ: 7069.08
  C4: 4741.66
  PO8: 2923.19
  PO7: 2572.96
  C3: 2323.86
  FZ: 1957.84

--- Recommended Top 3 Channels based on F-score: ['CZ', 'C4', 'PO8'] ---


In [67]:
# Example for train/val/test
X_train = np.stack([x.numpy() for x, y in dataset_train])  # shape: [N, C, T]
y_train = np.array([y[0] for x, y in dataset_train])

X_val = np.stack([x.numpy() for x, y in dataset_val])
y_val = np.array([y[0] for x, y in dataset_val])

X_test = np.stack([x.numpy() for x, y in dataset_test])
y_test = np.array([y[0] for x, y in dataset_test])

# Cheaty :)
X_train_full = np.concatenate([X_train, X_val], axis=0)
y_train_full = np.concatenate([y_train, y_val], axis=0)

In [68]:
pipeline = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("clf", FgMDM()),
    ]
)


pipeline.fit(X_train, y_train)
train_acc = accuracy_score(y_train, pipeline.predict(X_train))
val_acc = accuracy_score(y_val, pipeline.predict(X_val))

print(f"Train Acc = {train_acc:.3f} | Val Acc = {val_acc:.3f}")

Train Acc = 0.562 | Val Acc = 0.471


In [73]:
import numpy as np
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from pyriemann.estimation import Covariances
from pyriemann.utils.mean import mean_riemann
from pyriemann.tangentspace import TangentSpace
from scipy.linalg import inv, logm
from scipy.optimize import minimize


def fit_rsf(X, y, d, eps, maxiter=500):
    # 1) compute class means
    covs = Covariances(estimator="lwf").transform(X)
    C1 = mean_riemann(covs[y == 0])
    C2 = mean_riemann(covs[y == 1])
    Nc = C1.shape[0]
    W0 = np.random.randn(Nc, d)

    def J(w):
        W = w.reshape(Nc, d)
        S1 = W.T @ C1 @ W + eps * np.eye(d)
        S2 = W.T @ C2 @ W + eps * np.eye(d)
        return np.linalg.norm(logm(inv(S1) @ S2), "fro") ** 2

    def grad(w):
        W = w.reshape(Nc, d)
        S1 = W.T @ C1 @ W + eps * np.eye(d)
        S2 = W.T @ C2 @ W + eps * np.eye(d)
        A = inv(S1) @ S2
        L = logm(A)
        term1 = 2 * C1.dot(W).dot(inv(S1)).dot(L).dot(inv(S1))
        term2 = 2 * C2.dot(W).dot(inv(S2)).dot(L).dot(inv(S2))
        return (term1 - term2).ravel()

    res = minimize(lambda w: -J(w), W0.ravel(), jac=lambda w: -grad(w), method="BFGS", options={"maxiter": maxiter})
    return res.x.reshape(Nc, d)


def evaluate_rsf(X_train, y_train, X_val, y_val, d, eps, C_lr):
    W = fit_rsf(X_train, y_train, d, eps)
    # build the rest of the pipeline
    # 1) project
    Xf_train = np.einsum("cd,ect->edt", W, X_train)
    Xf_val = np.einsum("cd,ect->edt", W, X_val)
    # 2) covariances
    cov_train = Covariances("lwf").transform(Xf_train)
    cov_val = Covariances("lwf").transform(Xf_val)
    # 3) tangent‑space
    ts = TangentSpace()
    feat_train = ts.fit_transform(cov_train)
    feat_val = ts.transform(cov_val)
    # 4) classify
    clf = LogisticRegression(C=C_lr, max_iter=500)
    clf.fit(feat_train, y_train)
    return clf.score(feat_train, y_train), clf.score(feat_val, y_val)


# define your grid
grid_d = [4, 8, 12]
grid_eps = [1e-6, 1e-4, 1e-2]
grid_C = [0.01, 0.1, 1, 10]

best = {"score": -np.inf}
for d, eps, C_lr in product(grid_d, grid_eps, grid_C):
    tr, va = evaluate_rsf(X_train, y_train, X_val, y_val, d, eps, C_lr)
    print(f"d={d:2d}, eps={eps:.1e}, C={C_lr}: train={tr:.3f}, val={va:.3f}")
    if va > best["score"]:
        best = {"d": d, "eps": eps, "C": C_lr, "score": va}

print("Best config:", best)

d= 4, eps=1.0e-06, C=0.01: train=0.529, val=0.494
d= 4, eps=1.0e-06, C=0.1: train=0.546, val=0.506
d= 4, eps=1.0e-06, C=1: train=0.543, val=0.471
d= 4, eps=1.0e-06, C=10: train=0.540, val=0.447
d= 4, eps=1.0e-04, C=0.01: train=0.569, val=0.424
d= 4, eps=1.0e-04, C=0.1: train=0.554, val=0.459
d= 4, eps=1.0e-04, C=1: train=0.545, val=0.447
d= 4, eps=1.0e-04, C=10: train=0.540, val=0.494
d= 4, eps=1.0e-02, C=0.01: train=0.555, val=0.494
d= 4, eps=1.0e-02, C=0.1: train=0.537, val=0.435
d= 4, eps=1.0e-02, C=1: train=0.557, val=0.388
d= 4, eps=1.0e-02, C=10: train=0.522, val=0.471
d= 8, eps=1.0e-06, C=0.01: train=0.565, val=0.400
d= 8, eps=1.0e-06, C=0.1: train=0.565, val=0.412
d= 8, eps=1.0e-06, C=1: train=0.562, val=0.388
d= 8, eps=1.0e-06, C=10: train=0.568, val=0.435
d= 8, eps=1.0e-04, C=0.01: train=0.572, val=0.435
d= 8, eps=1.0e-04, C=0.1: train=0.568, val=0.424
d= 8, eps=1.0e-04, C=1: train=0.566, val=0.400
d= 8, eps=1.0e-04, C=10: train=0.572, val=0.424
d= 8, eps=1.0e-02, C=0.01: tra

In [69]:
covs = Covariances(estimator="lwf").transform(X_train)  # shape: (n_epochs, C, C)
# 1. Compute class‐mean covariances C1, C2 from your epochs X_epochs (shape: n_epochs × C × T)
C1 = mean_riemann(covs[y_train == 0])  # class 0 mean SPD
C2 = mean_riemann(covs[y_train == 1])  # class 1 mean SPD

Nc = C1.shape[0]  # number of channels
d = 8  # desired RSF dimension (e.g. 8)
W0 = np.random.randn(Nc, d)  # random initialization


# 2. Objective: maximize δ_R^2(WᵀC1W, WᵀC2W) :contentReference[oaicite:0]{index=0}
def J(W_flat):
    W = W_flat.reshape(Nc, d)
    eps = 1e-6
    S1 = W.T @ C1 @ W + np.eye(d) * eps
    S2 = W.T @ C2 @ W + np.eye(d) * eps
    # affine‐invariant Riemannian distance squared:
    return np.linalg.norm(logm(inv(S1) @ S2), "fro") ** 2


# 3. Gradient ∇J(W) from Eq. (5) :contentReference[oaicite:1]{index=1}
def grad_J(W_flat):
    W = W_flat.reshape(Nc, d)
    eps = 1e-6
    S1 = W.T @ C1 @ W + np.eye(d) * eps
    S2 = W.T @ C2 @ W + np.eye(d) * eps
    A = inv(S1) @ S2
    L = logm(A)
    # full formula is a bit long; here’s the structure:
    #    ∇J = 2 · C1 · W · S1^{-1} · L · S1^{-1}
    #         − 2 · C2 · W · S2^{-1} · L · S2^{-1}
    term1 = 2 * C1.dot(W).dot(inv(S1)).dot(L).dot(inv(S1))
    term2 = 2 * C2.dot(W).dot(inv(S2)).dot(L).dot(inv(S2))
    return (term1 - term2).ravel()


# 4. Run BFGS to maximize J (we minimize -J)
res = minimize(lambda w: -J(w), W0.ravel(), jac=lambda w: -grad_J(w), method="BFGS", options={"maxiter": 1000, "gtol": 1e-8})

W_opt = res.x.reshape(Nc, d)
print("Converged RSF W shape:", W_opt.shape)

Converged RSF W shape: (6, 8)


In [72]:

class RSFTransformer(BaseEstimator, TransformerMixin):
    """Project multichannel epochs X (n_epochs, C, T) to d channels via W_opt."""
    def __init__(self, W):
        self.W = W  # shape (C, d)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X: (n_epochs, C, T) → Xf: (n_epochs, d, T)
        return np.einsum('cd,ect->edt', self.W, X)

# Build the pipeline
rsf_pipe = make_pipeline(
    RSFTransformer(W_opt),                     # step 1: apply RSF
    Covariances(estimator='lwf'),              # step 2: covariances on d channels
    TangentSpace(metric='riemann'),            # step 3: tangent‑space embedding
    LogisticRegression(random_state=42, tol=0.0001, solver='sag', penalty='l2', max_iter=1000, class_weight='balanced', C=0.001)          # step 4: classifier
)

# Fit + evaluate
rsf_pipe.fit(X_train, y_train)
train_acc = rsf_pipe.score(X_train, y_train)
val_acc   = rsf_pipe.score(X_val,   y_val)

print(f"Train Acc = {train_acc:.3f} | Val Acc = {val_acc:.3f}")

Train Acc = 0.566 | Val Acc = 0.494


In [71]:
pipeline_svm = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("tangent", TangentSpace(metric="riemann")),
        ("clf", SVC(random_state=42, tol=0.001, kernel='rbf', gamma=0.01, class_weight='balanced', C=10)),
    ]
)

pipeline_lr = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("tangent", TangentSpace(metric="riemann")),
        ("clf", LogisticRegression(random_state=42, tol=0.0001, solver='sag', penalty='l2', max_iter=1000, class_weight='balanced', C=0.001)),
    ]
)


pipeline_mdm = Pipeline([
    ('cov', Covariances(estimator="lwf")),
    ("clf", MDM()),
])

pipelines = {
    "SVM": pipeline_svm,
    "Logistic Regression": pipeline_lr,
    "MDM": pipeline_mdm,
}

results = {}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, pipe.predict(X_train))
    val_acc = accuracy_score(y_val, pipe.predict(X_val))
    results[name] = (train_acc, val_acc)

print("=== Model Performance Report ===")
for name, (train_acc, val_acc) in results.items():
    print(f"{name:>20}: Train Acc = {train_acc:.3f} | Val Acc = {val_acc:.3f}")

=== Model Performance Report ===
                 SVM: Train Acc = 0.653 | Val Acc = 0.435
 Logistic Regression: Train Acc = 0.567 | Val Acc = 0.506
                 MDM: Train Acc = 0.515 | Val Acc = 0.518


In [ ]:
# SVM
# Best CV score:    0.5679161628375655
# Best parameters:  {'clf__tol': 0.001, 'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__class_weight': 'balanced', 'clf__C': 10}
# Validation accuracy: 0.5145502645502645

param_grid_svm = {
    "clf__kernel": ["rbf", "linear"],  # Drop poly - research shows overfitting
    "clf__C": [0.01, 0.1, 1, 10, 50],  # Add lower values
    "clf__gamma": [0.001, 0.01, 0.1, "scale"],  # Finer granularity
    "clf__class_weight": ["balanced"],
    "clf__tol": [1e-3, 1e-4],
    # Remove degree/coef0 (irrelevant for non-poly kernels)
}

pipeline = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("tangent", TangentSpace(metric="riemann")),
        ("clf", SVC(random_state=42)),
    ]
)

grid = RandomizedSearchCV(
    estimator=pipeline,
    n_iter=50,
    param_distributions=param_grid_svm,
    # param_grid=param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
)

grid.fit(X_train, y_train)

# 3) Inspect best params & CV score
print("Best CV score:   ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

# # 4) Evaluate on validation set
best_model = grid.best_estimator_
y_val_pred = best_model.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", val_acc)

In [ ]:
# Logistic Regression 
# Best CV score:    0.5507187961843343
# Best parameters:  {'clf__tol': 0.0001, 'clf__solver': 'sag', 'clf__penalty': 'l2', 'clf__max_iter': 1000, 'clf__class_weight': 'balanced', 'clf__C': 0.001}
# Validation accuracy: 0.5066137566137566

param_grid_lr = {
    "clf__penalty": ["l2", None],
    "clf__C": [0.001, 0.01, 0.1, 1, 10],
    "clf__solver": ["lbfgs", "sag"],
    "clf__max_iter": [1000],
    "clf__class_weight": ["balanced"],
    "clf__tol": [1e-4]
}

pipeline = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("tangent", TangentSpace(metric="riemann")),
        ("clf", LogisticRegression(random_state=42)),
    ]
)

grid_lr = RandomizedSearchCV(
    estimator=pipeline,
    n_iter=70,
    param_distributions=param_grid_lr,
    # param_grid=param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2,
)

grid_lr.fit(X_train, y_train)

# 3) Inspect best params & CV score
print("Best CV score:   ", grid_lr.best_score_)
print("Best parameters: ", grid_lr.best_params_)

# # 4) Evaluate on validation set
best_model = grid_lr.best_estimator_
y_val_pred = best_model.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", val_acc)

In [ ]:
# # MDM

# param_grid_mdm = {
#     "clf__metric": ["riemann"],
#     "clf__n_means": [3, 5, 7],  # Number of power means
#     "clf__h_values": [
#         [-1, 0, 1], 
#         [-0.5, 0, 0.5],
#         [-1, -0.2, 0.2, 1]
#     ],  # Power parameters
#     "clf__mean_type": ["power"]
# }

pipeline = Pipeline(
    [
        ("cov", Covariances(estimator="lwf")),
        ("clf", MDM(metric="riemann")),
    ]
)

# grid_mdm = RandomizedSearchCV(
#     estimator=pipeline,
#     n_iter=70,
#     param_distributions=param_grid_mdm,
#     # param_grid=param_grid,
#     cv=3,
#     scoring="accuracy",
#     n_jobs=-1,
#     verbose=2,
# )

pipeline.fit(X_train, y_train)

# 3) Inspect best params & CV score
# print("Best CV score:   ", pipeline.best_score_)
# print("Best parameters: ", pipeline.best_params_)

# # 4) Evaluate on validation set
# best_model = pipeline.best_estimator_
y_val_pred = pipeline.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", val_acc)